In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import_path = "/Users/michalmacbookpro/Downloads/titanic/"
train = pd.read_csv(import_path + "train.csv",index_col="PassengerId")
test = pd.read_csv(import_path + "test.csv",index_col="PassengerId")
submision = pd.read_csv(import_path + "gender_submission.csv")

In [3]:
train.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
def titulizer(df):
    df["Title"] = df["Name"].str.extract(r"([A-Za-z]+\.)")
    df["Title"].replace(['Capt.', 'Col.','Major.',"Master.",'Master.','Countess.', 'Don.', 'Dona.', 'Dr.', 'Jonkheer.', 'Lady.', 'Rev.', 'Sir.'], 'Rare.', inplace = True)
    df["Title"].replace(['Mlle.', 'Ms.'], 'Miss.', inplace = True)
    df["Title"].replace(['Mme.'], 'Mrs.', inplace = True)
    return df


#df_train.sample(10)
def cabinizer(df):
    df["Cabin_L"] = df["Cabin"].str.extract(r"(^\D{0,1})")
    df["Cabin_L"].fillna("U",inplace=True)
    df["Cabin_L"] = df["Cabin_L"].map({"A":"ABC","B":"ABC","C":"ABC","D":"DE","E":"DE","F":"FG","G":"FG","T":"ABC","U":"U"})
    return df


def agegrouping(df):
    df["Age_Group"] = pd.cut(df["Age"],bins=[0,17,33,49,65,83],labels=[1,2,3,4,5])
    return df

def faregrouping(df):
    df["Fare_Group"] = pd.cut(df["Fare"],bins=[-1,7,14,32,600],labels=[1,2,3,4])
    return df

def age_imp(df):
    df["Age"].fillna(np.mean(df["Age"]),inplace=True)
    return df

def familiazer(df):
    df["Family"] = df["SibSp"] + df["Parch"] + 1
    df["Family"] = df["Family"].astype("str")
    return df

In [5]:
combined =[train, test]
combined = pd.concat(combined,axis=0)

In [6]:
combined.isna().sum()

Survived     418
Pclass         0
Name           0
Sex            0
Age          263
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [7]:
combined["Age"].fillna(np.mean(combined["Age"]),inplace=True)
combined["Fare"].fillna(np.mean(combined["Fare"]),inplace=True)
combined["Embarked"].fillna("C",inplace=True)
combined.isna().sum()

Survived     418
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [8]:
titulizer(combined)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,Mr.
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,Mrs.
3,1.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,Mrs.
5,0.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,Mr.
...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",male,29.881138,0,0,A.5. 3236,8.0500,NaN,S,Mr.
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C,Rare.
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr.


In [9]:
combined["Title"].unique()

array(['Mr.', 'Mrs.', 'Miss.', 'Rare.'], dtype=object)

In [10]:
combined["Fare"].describe()

count    1309.000000
mean       33.295479
std        51.738879
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

In [11]:
agegrouping(combined)
faregrouping(combined)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Age_Group,Fare_Group
PassengerId,,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,Mr.,2,2
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,Mrs.,3,4
3,1.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,2,2
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,Mrs.,3,4
5,0.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,Mr.,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",male,29.881138,0,0,A.5. 3236,8.0500,NaN,S,Mr.,2,2
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C,Rare.,3,4
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr.,3,2


In [12]:
combined["Age_Group"].unique()

[2, 3, 4, 1, 5]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [13]:
combined["Fare_Group"].unique()

[2, 4, 3, 1]
Categories (4, int64): [1 < 2 < 3 < 4]

In [15]:
cabinizer(combined)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Age_Group,Fare_Group,Cabin_L
PassengerId,,,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,Mr.,2,2,U
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,Mrs.,3,4,ABC
3,1.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,2,2,U
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,Mrs.,3,4,ABC
5,0.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,Mr.,3,2,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",male,29.881138,0,0,A.5. 3236,8.0500,NaN,S,Mr.,2,2,U
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C,Rare.,3,4,ABC
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr.,3,2,U


In [16]:
train = combined.iloc[:len(train),:]

In [17]:
train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Age_Group,Fare_Group,Cabin_L
PassengerId,,,,,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,Mr.,2,2,U
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,Mrs.,3,4,ABC
3,1.0,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss.,2,2,U
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,Mrs.,3,4,ABC
5,0.0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,Mr.,3,2,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0.0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,Rare.,2,2,U
888,1.0,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,Miss.,2,3,ABC
889,0.0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.881138,1,2,W./C. 6607,23.4500,NaN,S,Miss.,2,3,U


In [18]:
test = combined.iloc[len(train):,:]
test

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Age_Group,Fare_Group,Cabin_L
PassengerId,,,,,,,,,,,,,,,
892,NaN,3,"Kelly, Mr. James",male,34.500000,0,0,330911,7.8292,NaN,Q,Mr.,3,2,U
893,NaN,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.000000,1,0,363272,7.0000,NaN,S,Mrs.,3,1,U
894,NaN,2,"Myles, Mr. Thomas Francis",male,62.000000,0,0,240276,9.6875,NaN,Q,Mr.,4,2,U
895,NaN,3,"Wirz, Mr. Albert",male,27.000000,0,0,315154,8.6625,NaN,S,Mr.,2,2,U
896,NaN,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.000000,1,1,3101298,12.2875,NaN,S,Mrs.,2,2,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",male,29.881138,0,0,A.5. 3236,8.0500,NaN,S,Mr.,2,2,U
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C,Rare.,3,4,ABC
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr.,3,2,U


In [19]:
y = train["Survived"].copy()

In [20]:
to_drop =["Survived","Name","Sex", "Age", "Ticket", "Fare",]
combined.drop(to_drop,inplace=True,axis=1)

In [21]:
combined.drop(["Cabin"],inplace=True,axis=1)

In [24]:
combined["Family_size"] = combined["Parch"] + combined["SibSp"]

In [25]:
combined["Family_size"].describe()

count    1309.000000
mean        0.883881
std         1.583639
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        10.000000
Name: Family_size, dtype: float64

In [26]:
combined["Alone"] = 0

In [27]:
combined["Family_size"]>1

PassengerId
1       False
2       False
3       False
4       False
5       False
        ...  
1305    False
1306    False
1307    False
1308    False
1309     True
Name: Family_size, Length: 1309, dtype: bool

In [28]:
combined.loc[combined["Family_size"]==0,"Alone"]= 1

In [29]:
combined

,Pclass,SibSp,Parch,Embarked,Title,Age_Group,Fare_Group,Cabin_L,Family_size,Alone
PassengerId,,,,,,,,,,
1,3,1,0,S,Mr.,2,2,U,1,0
2,1,1,0,C,Mrs.,3,4,ABC,1,0
3,3,0,0,S,Miss.,2,2,U,0,1
4,1,1,0,S,Mrs.,3,4,ABC,1,0
5,3,0,0,S,Mr.,3,2,U,0,1
...,...,...,...,...,...,...,...,...,...,...
1305,3,0,0,S,Mr.,2,2,U,0,1
1306,1,0,0,C,Rare.,3,4,ABC,0,1
1307,3,0,0,S,Mr.,3,2,U,0,1


In [30]:
combined["Family_size"].value_counts()

0     790
1     235
2     159
3      43
5      25
4      22
6      16
10     11
7       8
Name: Family_size, dtype: int64

In [31]:
to_drop_2 = ["SibSp","Parch","Family_size"]

In [32]:
combined.drop(to_drop_2,axis=1,inplace=True)

In [33]:
combined

,Pclass,Embarked,Title,Age_Group,Fare_Group,Cabin_L,Alone
PassengerId,,,,,,,
1,3,S,Mr.,2,2,U,0
2,1,C,Mrs.,3,4,ABC,0
3,3,S,Miss.,2,2,U,1
4,1,S,Mrs.,3,4,ABC,0
5,3,S,Mr.,3,2,U,1
...,...,...,...,...,...,...,...
1305,3,S,Mr.,2,2,U,1
1306,1,C,Rare.,3,4,ABC,1
1307,3,S,Mr.,3,2,U,1


In [34]:
combined[["C","Q","S"]] = pd.get_dummies(combined["Embarked"])
combined

,Pclass,Embarked,Title,Age_Group,Fare_Group,Cabin_L,Alone,C,Q,S
PassengerId,,,,,,,,,,
1,3,S,Mr.,2,2,U,0,0,0,1
2,1,C,Mrs.,3,4,ABC,0,1,0,0
3,3,S,Miss.,2,2,U,1,0,0,1
4,1,S,Mrs.,3,4,ABC,0,0,0,1
5,3,S,Mr.,3,2,U,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1305,3,S,Mr.,2,2,U,1,0,0,1
1306,1,C,Rare.,3,4,ABC,1,1,0,0
1307,3,S,Mr.,3,2,U,1,0,0,1


In [35]:
combined[np.unique(combined["Title"])] = pd.get_dummies(combined["Title"])

In [36]:
combined

,Pclass,Embarked,Title,Age_Group,Fare_Group,Cabin_L,Alone,C,Q,S,Miss.,Mr.,Mrs.,Rare.
PassengerId,,,,,,,,,,,,,,
1,3,S,Mr.,2,2,U,0,0,0,1,0,1,0,0
2,1,C,Mrs.,3,4,ABC,0,1,0,0,0,0,1,0
3,3,S,Miss.,2,2,U,1,0,0,1,1,0,0,0
4,1,S,Mrs.,3,4,ABC,0,0,0,1,0,0,1,0
5,3,S,Mr.,3,2,U,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,S,Mr.,2,2,U,1,0,0,1,0,1,0,0
1306,1,C,Rare.,3,4,ABC,1,1,0,0,0,0,0,1
1307,3,S,Mr.,3,2,U,1,0,0,1,0,1,0,0


In [37]:
combined[np.unique(combined["Cabin_L"])] = pd.get_dummies(combined["Cabin_L"])

In [38]:
combined

,Pclass,Embarked,Title,Age_Group,Fare_Group,Cabin_L,Alone,C,Q,S,Miss.,Mr.,Mrs.,Rare.,ABC,DE,FG,U
PassengerId,,,,,,,,,,,,,,,,,,
1,3,S,Mr.,2,2,U,0,0,0,1,0,1,0,0,0,0,0,1
2,1,C,Mrs.,3,4,ABC,0,1,0,0,0,0,1,0,1,0,0,0
3,3,S,Miss.,2,2,U,1,0,0,1,1,0,0,0,0,0,0,1
4,1,S,Mrs.,3,4,ABC,0,0,0,1,0,0,1,0,1,0,0,0
5,3,S,Mr.,3,2,U,1,0,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3,S,Mr.,2,2,U,1,0,0,1,0,1,0,0,0,0,0,1
1306,1,C,Rare.,3,4,ABC,1,1,0,0,0,0,0,1,1,0,0,0
1307,3,S,Mr.,3,2,U,1,0,0,1,0,1,0,0,0,0,0,1


In [39]:
to_drop_3 = ["Embarked", "Title", "Cabin_L"]

In [40]:
combined.drop(to_drop_3,axis=1,inplace=True)

In [41]:
combined.sample(5)

,Pclass,Age_Group,Fare_Group,Alone,C,Q,S,Miss.,Mr.,Mrs.,Rare.,ABC,DE,FG,U
PassengerId,,,,,,,,,,,,,,,
701,1,2,4,0,1,0,0,0,0,1,0,1,0,0,0
531,2,1,3,0,0,0,1,1,0,0,0,0,0,0,1
863,1,3,3,1,0,0,1,0,0,1,0,0,1,0,0
571,2,4,2,1,0,0,1,0,1,0,0,0,0,0,1
1056,2,3,2,1,0,0,1,0,0,0,1,0,0,0,1


In [42]:
test = combined.iloc[len(train):,:]
train = combined.iloc[:len(train),:]


In [43]:
print(f"{train.shape},{y.shape},{test.shape}")

(891, 15),(891,),(418, 15)


In [44]:
from sklearn.model_selection import train_test_split

In [58]:
X_train,X_test,y_train,y_test = train_test_split(train,y,test_size=0.3,stratify=y,random_state=42)
X_train.drop(["Q"],axis=1,inplace=True)

In [65]:
X_test.drop(["Q"],axis=1,inplace=True)

In [109]:
X_test.drop(["Age_Group"],axis=1,inplace=True)
X_train.drop(["Age_Group"],axis=1,inplace=True)

In [127]:
X_test.drop(["C","S"],axis=1,inplace=True)
X_train.drop(["C","S"],axis=1,inplace=True)

In [147]:
X_test.drop(["ABC","DE","FG","U"],axis=1,inplace=True)
X_train.drop(["ABC","DE","FG","U"],axis=1,inplace=True)

In [148]:
X_train.corrwith(y_train,axis=0)

Pclass       -0.335331
Fare_Group    0.314856
Alone        -0.203274
Miss.         0.350782
Mr.          -0.560195
Mrs.          0.337313
Rare.         0.065972
dtype: float64

In [149]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

In [150]:
rfc = RandomForestClassifier()
dtc = DecisionTreeClassifier()

In [151]:
X_train["Fare_Group"].unique()

array([4, 2, 1, 3])

In [152]:
param = [{
    "criterion": ['gini', 'entropy'],
    "n_estimators":[100,200,300,400,500,600],
    'max_depth': range(4, 20, 2),
    'max_leaf_nodes': range(4, 28, 2),
}]

gs_rt = RandomizedSearchCV(rfc, param, cv = 5, n_jobs = -1, verbose = 1)
gs_rt.fit(X_train, y_train)
y_pred = gs_rt.predict(X_test)
print(gs_rt.best_estimator_)
print(accuracy_score(y_pred,y_test))
#rt_rs_best = gs_rt.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
RandomForestClassifier(max_depth=14, max_leaf_nodes=18, n_estimators=200)
0.7873134328358209


In [153]:
rfc_best = RandomForestClassifier(criterion="entropy", max_depth=16, max_leaf_nodes=22, n_estimators=400)

rfc_best.fit(X_train,y_train)
y_pred_2 = rfc_best.predict(X_test)
print(accuracy_score(y_pred_2,y_test))

0.7873134328358209


In [154]:
param = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': range(2, 20, 2),
        'max_leaf_nodes': range(4, 28, 2),
    }
]

gs_dt = GridSearchCV(dtc, param, cv = 5, n_jobs = -1, verbose = 1,scoring="accuracy")
gs_dt.fit(X_train, y_train)
y_pred = gs_dt.predict(X_test)
print(accuracy_score(y_pred,y_test))
#dt_gs_best = gs_dt.best_estimator_
print(gs_dt.best_estimator_)
#print(f1_score(y_pred,y_test))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
0.7873134328358209
DecisionTreeClassifier(max_depth=4, max_leaf_nodes=8)


In [155]:
dtc_best = RandomForestClassifier(criterion="entropy", max_depth=8, max_leaf_nodes=14)

dtc_best.fit(X_train,y_train)
y_pred_3 = dtc_best.predict(X_test)
print(accuracy_score(y_pred_3,y_test))

0.7873134328358209


In [156]:
from sklearn.svm import SVC

svc = SVC()

param = [
    {
        'kernel': ['poly',"rbf"], 'C': [0.1, 0.3, 1, 2, 3, 4], 
        'gamma': [0.1,0.2,0.3]
    }, 
]

svc = SVC(probability = True)
rs_svc = GridSearchCV(svc, param, cv =5, n_jobs = -1, verbose = 1)
rs_svc.fit(X_train, y_train)
y_pred = rs_svc.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(rs_svc.best_estimator_)
print(accuracy_score(y_pred,y_test))
svc_best = rs_svc.best_estimator_


Fitting 5 folds for each of 36 candidates, totalling 180 fits
0.7873134328358209
SVC(C=2, gamma=0.3, probability=True)
0.7873134328358209


In [137]:
svc_best = SVC(C=2, gamma=0.1, kernel='rbf', probability=True)
svc_best.fit(X_train,y_train)
y_pred_4 = svc_best.predict(X_test)
print(accuracy_score(y_pred_4,y_test))

0.7649253731343284


In [157]:
param = [
    {'n_estimators': range(8, 28, 4), 
     'max_depth': range(24, 48, 4),
     'max_leaf_nodes': range(20, 48, 4),
    }
]

et = ExtraTreesClassifier()
gs_et = RandomizedSearchCV(et, param, cv = 5, n_jobs = -1, verbose = 1)
gs_et.fit(X_train, y_train)
y_pred = gs_et.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(gs_et.best_estimator_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.7873134328358209
ExtraTreesClassifier(max_depth=24, max_leaf_nodes=36, n_estimators=16)


In [139]:
xtree_best = gs_et.best_estimator_
xtree_best.fit(X_train,y_train)
y_pred_5 = xtree_best.predict(X_test)
print(accuracy_score(y_pred_5,y_test))

0.7835820895522388


In [158]:
X_train["Fare_Group"] = X_train["Fare_Group"].astype("int")

In [159]:
#X_train["Age_Group"] = X_train["Age_Group"].astype("int")

In [160]:
param_grid={
    'max_depth': range(2, 10, 2),
    'n_estimators': range(26, 46, 2),
    'learning_rate': [0.2, 0.1, 0.03, 0.01]
}

xg = XGBClassifier(eval_metric='logloss', n_jobs = -1, use_label_encoder = False)
gs_xg = RandomizedSearchCV(xg, param_grid, cv = 5, n_jobs = -1, verbose = 1)
gs_xg.fit(X_train, y_train)
y_pred = gs_xg.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(gs_xg.best_estimator_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype inste

0.7835820895522388
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=32, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)


In [161]:
xboo_best = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1,
              monotone_constraints='()', n_estimators=26, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)
xboo_best.fit(X_train,y_train)
y_pred_6 = xboo_best.predict(X_test)
print(accuracy_score(y_pred_6,y_test))

0.7835820895522388


/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [162]:
voting_clf = VotingClassifier(estimators=[
    ("rand_for",rfc_best),
    ("dec_tree",dtc_best),
    ("svc",svc_best),
    ("extree",xtree_best),
    ("xgb",xboo_best)],voting = 'soft', n_jobs = -1, verbose = 1)
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(accuracy_score(y_pred,y_test))

/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[Voting] ................... (4 of 5) Processing extree, total=   0.0s
[Voting] ...................... (3 of 5) Processing svc, total=   0.1s
[Voting] ................. (2 of 5) Processing dec_tree, total=   0.1s
[Voting] ...................... (5 of 5) Processing xgb, total=   0.1s
[Voting] ................. (1 of 5) Processing rand_for, total=   0.3s
0.7873134328358209
